In [2]:
#importing libraries
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from time import sleep
import re
import requests
from bs4 import  BeautifulSoup
from bs4 import SoupStrainer
import pandas as pd
from urllib.request import urlopen
from fake_useragent import UserAgent
import lxml
import pandas as pd
import time
import json
import urllib
import datetime as dt
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [4]:
'Create instance of chrome'
chrome_path = r"C:\Users\kitty\OneDrive\Desktop\chromedriver\chromedriver.exe"
base_url="https://www.zomato.com/pune/restaurants?page=1"
driver = webdriver.Chrome(r"C:\Users\kitty\OneDrive\Desktop\chromedriver\chromedriver.exe")
driver.get(base_url)

In [10]:
records = []
def get_link(next_url, count):
    rests  = 0
    #time.sleep(10)
   
    rest = requests.get(next_url, headers = headers)
    soup_rest = BeautifulSoup(rest.content, 'html.parser')
    result = soup_rest.find_all('div', {'class' : 'col-s-12' })
    #print(result)
    for div in result:
        children = div.findChildren("a" , recursive=False)[0]

        link = (children['href'])
        print(link)
        rest_name = div.find("a" , recursive=False).text
        print(rest_name)


        'Restaurant Type'
        if div.find('div', {'class': 'res-snippet-small-establishment mt5'}) is not None:
            rest_type = div.find('div', {'class': 'res-snippet-small-establishment mt5'}).text
            #print(rest_type)

        else:
            rest_type = "NA"

        'Restaurant location '

        if div.find('a', {'class': 'ln24 search-page-text mr10 zblack search_result_subzone left'}) is not None:
            loc = div.find('a', {'class': 'ln24 search-page-text mr10 zblack search_result_subzone left'}).text
            #print(loc)

        else:
            loc = "NA"


        'Restaurant Dining Rating '
        x = div.find_all('span', {'class': 'rating-value'}) 
        
        if div.find('span', {'class': 'rating-value'}) is not None  and (len(x) > 0):
            dine_rating = div.find_all('span', {'class': 'rating-value'})[0].text
            #print(dine_rating)

        else:
            dine_rating = "NA"


        'Restaurant Dining Reviews'

        if div.find_all('span', {'class': 'review-count medium'}) is not None and (len(x) > 0):
            dine_reviews = div.find_all('span', {'class': 'review-count medium'})[0].text
            #print(dine_reviews)

        else:
            dine_reviews = "NA"


        'Restaurant Deliver reviews'   
        if div.find_all('span', {'class': 'rating-value '}) is not None and (len(x) > 1):
            delivery_rating = div.find_all('span', {'class': 'rating-value '})[1].text
            #print(delivery_rating)

        else:
            delivery_rating = "NA"


        'Restaurant Delivery Reviews'

        if div.find_all('span', {'class': 'review-count medium'}) is not None and (len(x) > 1):
            delivery_reviews = div.find_all('span', {'class': 'review-count medium'})[1].text
            #print(delivery_reviews)

        else:
            delivery_reviews = "NA"
            
            
        info =  soup_rest.find_all('div', {'class' : 'search-page-text clearfix row'})[rests]
        
        cuisine = ""
        cuisine_info = info.find('div', {'class' : 'clearfix'})
        if cuisine_info is not None and (len(cuisine_info) > 0): 
            cuisines = cuisine_info.find_all('a')

            for c in cuisines :
                cuisine  = c.text + ',' + cuisine 

            #print(cuisine)

        else:
            cuisine = "NA"

        if info.find('span', {'class' :'col-s-11 col-m-12 pl0' }) is not None:
            cost = info.find('span', {'class' :'col-s-11 col-m-12 pl0' }).text
        else:
            cost = "NA"
         
        #print(cost)
            
       
            

        rests = rests+1



        liked = get_data(link)
        records.append((link, rest_name,rest_type, loc,dine_rating, dine_reviews, delivery_rating, delivery_reviews, cuisine, cost, liked))

       
   
     
   
    

    outlets = soup_rest.find_all('a', {'class' :'ui ta-right pt10 fontsize3 zred pb10 pr10'})       
    
    if outlets is not None:
        
        for o in outlets:
            outlet = "https://www.zomato.com"+ o['href']
            print("outlet",outlet)
            
            get_link(outlet,1)
            #print(soup_rest.prettify)
            
            
    ''' This code is to get link for restaurants having multiple outlet pages... Like Dominos have 5 pages for outlets
      
      Count is passed otherwise it will result in an infinite loop Example - 
      1. First link----- >https://www.zomato.com/pune/restaurants?page=2-------> After this link we see that Dominos have SEE MORE OUTLETS
      -----> Href : https://www.zomato.com/pune/restaurants?cid=10506  Get link is called on this page to get link of all Dominos restaurants
    2. After this from below code, we check if there are multiple pages for outlets.. If yes, then below code will be activated.
    3. If count = 0 is not passed, then recursion function will  continuosly go to  1st page of outlet (https://www.zomato.com/pune/restaurants?cid=10506 -- for dominos)
      and thus recursion will run infinitely as it will always find  next page for few restaurants like dominos
    ''' 
            
    if (count==1):
        
        next_page_outlet = soup_rest.find('div', {'class': ' ui pagination menu small pagination-control res-menu-paginator'}).find_all('a', href=re.compile("cid"), title = re.compile("Go to")) 
        if next_page_outlet is not None:
            for n in range(1, len(next_page_outlet)):
                outlet2 = "https://www.zomato.com"+ next_page_outlet[n]['href']
                print("outlet 2",outlet2)
                
                'count = 0 makes sure this function does not run .. As when we get link of outlet pages.. we want this part of code to stop'
                get_link(outlet2, 0)
            
            
    
    
    
    
    
    

In [11]:
def get_data(link):
    
    data = []
    #for index,link in c['link'].head(n=2).iteritems():
        #time.sleep(3)
        
        
        
    
    driver.get(link)
    
    
    
    liked = ""
    try:
        liked = driver.find_element_by_xpath('//h5[text() ="Top Dishes People Order"]/following-sibling::p').text
        #print(liked)
    except NoSuchElementException:
        liked = "NA"
        
    return liked
    
    
    

In [12]:
for i in range(1,3):
    #next_page = f"https://www.zomato.com/pune/restaurants?sort=cd&page={i}"
    next_page = f"https://www.zomato.com/pune/restaurants?page={i}"
    
    print(i)
    ua = UserAgent()
    headers = {"Host": "www.zomato.com",
      "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36",
       "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
      "Accept-Language": "en-US,en;q=0.5",
      "Accept-Encoding": "gzip, deflate, br",
     "Referer": "https://www.zomato.com/",
      "Connection": "keep-alive"}
    
   
    time.sleep(30)
    get_link(next_page, 1)
    
    

1
https://www.zomato.com/pune/kayani-bakery-east-street
Kayani Bakery
                                    
https://www.zomato.com/pune/kfc-baner
KFC
                                    
https://www.zomato.com/pune/mcdonalds-chinchwad
McDonald's
                                    
https://www.zomato.com/pune/vaishali-fc-road
Vaishali
                                    
https://www.zomato.com/pune/madinah-restaurant-kondhwa
Madinah Restaurant
                                    
https://www.zomato.com/pune/le-plaisir-deccan-gymkhana
Le Plaisir
                                    
https://www.zomato.com/pune/sps-biryani-house-sadashiv-peth
SP's Biryani House
                                    
https://www.zomato.com/pune/blue-nile-camp-area
Blue Nile
                                    
https://www.zomato.com/pune/burger-king-kothrud
Burger King
                                    
https://www.zomato.com/pune/moshi-moshi-koregaon-park
Moshi Moshi
                                    
ht

https://www.zomato.com/pune/burger-king-hadapsar
Burger King
                                    
https://www.zomato.com/pune/burger-king-1-senapati-bapat-road
Burger King
                                    
2
https://www.zomato.com/pune/brahma-pure-veg-sinhgad-road
Brahma Pure Veg
                                    
https://www.zomato.com/pune/samudra-erandwane
Samudra
                                    
https://www.zomato.com/pune/george-restaurant-camp-area
George Restaurant
                                    
https://www.zomato.com/pune/panchami-pure-veg-parvati
Panchami Pure Veg
                                    
https://www.zomato.com/pune/theobroma-koregaon-park
Theobroma
                                    
https://www.zomato.com/pune/filli-bites-nibm-road
Filli Bites
                                    
https://www.zomato.com/pune/garden-vada-pav-centre-erandwane
Garden Vada Pav Centre
                                    
https://www.zomato.com/pune/pk-biryani-house-3-sw

In [8]:

rest = pd.DataFrame(records, columns = ['link', 'rest_name','rest_type', 'loc','dine_rating', 'dine_reviews', 'delivery_rating', 'delivery_reviews', 'cuisine', 'cost', 'liked'])
rest.head()

,link,rest_name,rest_type,loc,dine_rating,dine_reviews,delivery_rating,delivery_reviews,cuisine,cost,liked
0,https://www.zomato.com/pune/kayani-bakery-east...,Kayani Bakery\n ...,Bakery,East Street,4.4,"(2,876 Reviews)",NA,NA,"Bakery,",₹150,"Mawa Cake, Mava Cake, Shrewsberry Biscuit, Cho..."
1,https://www.zomato.com/pune/kfc-baner,KFC\n,Quick Bites,Baner,2.5,(770),3.9,(13.6K),"Beverages,Finger Food,Fast Food,Burger,",₹450,"Burgers, Chicken Popcorn, Hot Wings, Chicken F..."
2,https://www.zomato.com/pune/vaishali-fc-road,Vaishali\n,Casual Dining,FC Road,4.5,"(5,662)",4.5,(10.2K),"Beverages,Fast Food,Sandwich,South Indian,",₹500,"Mysore Masala Dosa, Filtered Coffee, Idli, Upm..."
3,https://www.zomato.com/pune/madinah-restaurant...,Madinah Restaurant\n ...,Casual Dining,Kondhwa,4.1,"(1,140)",4.0,(22.1K),"Mughlai,Chinese,North Indian,",₹400,"Chicken Biryani, Murgh Angara, Paratha, Tandoo..."
4,https://www.zomato.com/pune/le-plaisir-deccan-...,Le Plaisir\n,Café,Deccan Gymkhana,4.9,"(4,742)",4.4,"(2,766)","Cafe,Continental,Italian,European,","₹1,000","Pasta, Sandwich, Macaroon, Cheesecake, Salad, ..."


In [9]:
rest['rest_name'].nunique()

32

In [42]:
zomato_outlets =rest.to_csv('C:/Users/kitty/OneDrive/Desktop/ML Practise/zomato_outlet.csv' ,index = None, header=True)